In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
sys.path.append("../")
from utils.load_brad_trace import load_trace, create_concurrency_dataset, load_trace_all_version
from models.concurrency.linear_regression import SimpleLinearReg
from models.concurrency.xgboost import XGBoostPredictor
np.set_printoptions(suppress=True)

In [3]:
#directory = "/Users/ziniuw/Desktop/research/Data/BRAD_trace/brad_dl_traces/brad_del_trace.task.1706423438/trace/"
#directory = "/Users/ziniuw/Desktop/research/Data/BRAD_trace/brad_dl_traces/brad_del_trace.task.1706488009/trace/"
directory = "/Users/ziniuw/Desktop/research/Data/AWS_trace/concurrent_3_brad/"
directory = "/Users/ziniuw/Desktop/research/Data/AWS_trace/io_heavy_redshift/"
raw_trace, trace = load_trace(directory, concat=True)
concurrency_df = create_concurrency_dataset(trace, engine=None, pre_exec_interval=60)
isolated_trace_df = pd.read_csv("/Users/ziniuw/Desktop/research/Data/BRAD_trace/io_heavy_redshift/repeating_olap_batch_warmup.csv")
isolated_trace_df["runtime"] = isolated_trace_df["run_time_s"]
isolated_rt_cache = dict()
for i, rows in isolated_trace_df.groupby("query_idx"):
    isolated_rt_cache[i] = np.median(rows["runtime"])

FileNotFoundError: [Errno 2] No such file or directory: '/Users/ziniuw/Desktop/research/Data/BRAD_trace/io_heavy_redshift/repeating_olap_batch_warmup.csv'

In [48]:
folder_name = "mixed_aurora"
directory = f"/Users/ziniuw/Desktop/research/Data/AWS_trace/{folder_name}/"
all_raw_trace, all_trace = load_trace_all_version(directory, 8, concat=True)
all_concurrency_df = []
for trace in all_trace:
    concurrency_df = create_concurrency_dataset(trace, engine=None, pre_exec_interval=60)
    all_concurrency_df.append(concurrency_df)
concurrency_df = pd.concat(all_concurrency_df, ignore_index=True)
isolated_trace_df = pd.read_csv(f"/Users/ziniuw/Desktop/research/Data/AWS_trace/{folder_name}/repeating_olap_batch_warmup.csv")
isolated_trace_df["runtime"] = isolated_trace_df["run_time_s"]
isolated_rt_cache = dict()
for i, rows in isolated_trace_df.groupby("query_idx"):
    isolated_rt_cache[i] = np.median(rows["runtime"])

client 4 trace not found
client 5 trace not found
client 6 trace not found
client 7 trace not found
client 4 trace not found
client 5 trace not found
client 6 trace not found
client 7 trace not found
client 4 trace not found
client 5 trace not found
client 6 trace not found
client 7 trace not found
client 4 trace not found
client 5 trace not found
client 6 trace not found
client 7 trace not found


In [ ]:
for i, rows in isolated_trace_df.groupby("query_idx"):
    runtime = rows["runtime"].values
    print(i, len(rows), np.mean(runtime), np.min(runtime), np.max(runtime), np.std(runtime))

In [49]:
for i, rows in concurrency_df.groupby("query_idx"):
    runtime = rows["runtime"].values
    print(i, len(rows), isolated_rt_cache[i], np.mean(runtime), np.min(runtime), np.max(runtime), np.std(runtime))
    #if i == 10:
     #   break
    #plt.hist(runtime, bins=60)
    #plt.xlabel("Query runtime (s)", fontsize=20)
    #plt.ylabel("Number of queries", fontsize=20)
    #plt.savefig(f"query_{i}.pdf")
    #plt.show()

0 192 44.55030298233032 138.26840952908 3.882469892501831 928.3928072452544 145.34024316926795
1 193 0.2543864250183105 0.643088676768881 0.0791635513305664 7.855563879013061 1.06857356261634
2 194 23.157404899597168 154.48734574342512 6.902190923690796 949.1584985256196 154.43707035632832
3 197 0.283794641494751 0.3924284259679959 0.0799896717071533 10.761748790740969 0.8463170517387892
4 194 4.330385208129883 20.928600831130115 0.291949987411499 173.76991820335388 28.976760172721317
5 196 2.5214909315109253 15.358369459911268 2.4990737438201904 183.3675537109375 26.940950509164374
6 194 2.5594241619110107 3.6193096416512716 2.4973695278167725 18.826606512069706 1.9008486815505048
7 195 2.2192370891571045 11.361962952980628 1.7622826099395752 173.82364749908447 24.60822146206646
8 196 0.3999779224395752 1.2696733973464187 0.1878495216369629 19.547641038894653 2.7639172885730847
9 197 0.7344051599502563 0.11367139114341152 0.0426087379455566 8.005128622055054 0.5712909656804914
10 194 

In [50]:
np.random.seed(4)
train_idx = np.random.choice(len(concurrency_df), size=int(0.8 * len(concurrency_df)), replace=False)
test_idx = [i for i in range(len(concurrency_df)) if i not in train_idx]
eval_trace_df = concurrency_df.iloc[test_idx]
eval_trace_df = eval_trace_df[eval_trace_df['num_concurrent_queries'] > 0]

slr = SimpleLinearReg()
slr.train(concurrency_df.iloc[train_idx], isolated_trace_df=isolated_trace_df)
#predictions, labels = slr.predict(eval_trace_df)
#print("===========Performance for simple linear regression model (per query)=============")
#result_overall, result_per_query = slr.evaluate_performance(eval_trace_df, use_train=True)
predictions_slr, labels = slr.predict(eval_trace_df, use_global=True)
print("===========Performance for simple linear regression model (all query)=============")
result_overall_slr, result_per_query_slr = slr.evaluate_performance(eval_trace_df, use_global=True)

===========Performance for simple linear regression model (all query)=============
50% absolute error is 18.639718668566466, q-error is 3.859273619392092
90% absolute error is 135.40292043812838, q-error is 10.336109159102925
95% absolute error is 212.96806786203737, q-error is 15.94459224910284


In [51]:
slr = SimpleLinearReg()
slr.train(concurrency_df.iloc[train_idx], use_train=False, isolated_trace_df=isolated_trace_df)
#predictions, labels = slr.predict(eval_trace_df, use_train=False)
#print("===========Performance for simple linear regression model (per query & train on Full)=============")
#result_overall, result_per_query = slr.evaluate_performance(eval_trace_df, use_train=False)
predictions_slr, labels = slr.predict(eval_trace_df, use_global=True)
print("===========Performance for simple linear regression model (all query & train on Full)=============")
result_overall_slr, result_per_query_slr = slr.evaluate_performance(eval_trace_df, use_global=True)

===========Performance for simple linear regression model (all query & train on Full)=============
50% absolute error is 16.179458680060897, q-error is 3.6733979295914096
90% absolute error is 156.9115795848445, q-error is 9.301906359707496
95% absolute error is 324.18740574361726, q-error is 13.571896883271435


In [52]:
sfc = SimpleFitCurve()
sfc.train(concurrency_df.iloc[train_idx], isolated_trace_df=isolated_trace_df)
#predictions, labels = sfc.predict(eval_trace_df)
#print("===========Performance for simple curve fitting model (per query)=============")
#result_overall, result_per_query = sfc.evaluate_performance(eval_trace_df, use_train=True)
predictions_sfc, labels = sfc.predict(eval_trace_df, use_global=True)
print("===========Performance for simple curve fitting model (all query)=============")
result_overall_sfc, result_per_query_sfc = sfc.evaluate_performance(eval_trace_df, use_global=True)

===========Performance for simple curve fitting model (all query)=============
50% absolute error is 9.717477197577608, q-error is 2.790340732361483
90% absolute error is 104.48327758084693, q-error is 7.701319729796892
95% absolute error is 166.8633528354039, q-error is 11.950354088544245


/Users/ziniuw/.local/lib/python3.11/site-packages/scipy/optimize/_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


In [53]:
sfc = SimpleFitCurve()
sfc.train(concurrency_df.iloc[train_idx], use_train=False, isolated_trace_df=isolated_trace_df)
#predictions, labels = sfc.predict(eval_trace_df, use_train=False)
#print("===========Performance for simple curve fitting model (per query)=============")
predictions_sfc, labels = sfc.predict(eval_trace_df, use_global=True)
print("===========Performance for simple curve fitting model (all query & train on full)=============")
result_overall_sfc, result_per_query_sfc = sfc.evaluate_performance(eval_trace_df, use_global=False)

===========Performance for simple curve fitting model (all query & train on full)=============
50% absolute error is 11.776617109778439, q-error is 2.2926540648477447
90% absolute error is 70.3883388328106, q-error is 7.333027856304992
95% absolute error is 108.4684800024337, q-error is 10.772560314095534


In [54]:
import numpy as np
from models.concurrency.base_model import ConcurPredictor
from xgboost import XGBRegressor


class XGBoostPredictor(ConcurPredictor):
    """
    Consider k unique concurrent queries (or group into k classes)
    For each query represent it as k-dimensional one hot vector
         Then, identify its concurrent queries, represent them as a k-dimensional vector
         [rt_q1, rt_q2, ..., rt_qk], where rt_qi is the runtime of query i running in isolation
         Concatenate two features together to build simple XGboost model to predict the runtime
    """
    def __init__(self, k=100):
        self.clustering = None
        self.isolated_rt_cache = dict()
        self.xgboost = None
        self.use_train = True
        self.use_pre_exec_info = False
        self.k = k

    def train(self, trace_df, use_train=True, isolated_trace_df=None, use_pre_exec_info=False):
        self.use_train = use_train
        self.use_pre_exec_info = use_pre_exec_info
        self.get_isolated_runtime_cache(trace_df, isolated_trace_df)
        concurrent_df = trace_df[trace_df['num_concurrent_queries'] > 0]

        global_y = []
        global_x = []
        for i, rows in concurrent_df.groupby("query_idx"):
            if i not in self.isolated_rt_cache or len(rows) < 2:
                continue
            if use_train:
                concur_info = rows["concur_info_train"].values
            else:
                concur_info = rows["concur_info"].values
            global_y.append(rows["runtime"].values)
            query_feature = np.zeros((len(rows), self.k))
            query_feature[:, i] = self.isolated_rt_cache[i]
            #query_feature[:, i] = 1
            concur_query_feature = np.zeros((len(rows), self.k))
            for j in range(len(rows)):
                for c in concur_info[j]:
                    if c[0] in self.isolated_rt_cache:
                        concur_query_feature[j, c[0]] += self.isolated_rt_cache[c[0]]
                        #concur_query_feature[j, c[0]] += 1
                    else:
                        concur_query_feature[j, c[0]] += 1
            x = np.concatenate((query_feature, concur_query_feature), axis=1)
            if use_pre_exec_info:
                pre_exec_query_feature = np.zeros((len(rows), self.k))
                pre_exec_info = rows["pre_exec_info"].values
                for j in range(len(rows)):
                    for c in pre_exec_info[j]:
                        if c[0] in self.isolated_rt_cache:
                            pre_exec_query_feature[j, c[0]] += self.isolated_rt_cache[c[0]]
                            #concur_query_feature[j, c[0]] += 1
                        else:
                            pre_exec_query_feature[j, c[0]] += 1
                x = np.concatenate((x, pre_exec_query_feature), axis=1)
            global_x.append(x)
        global_y = np.concatenate(global_y)
        global_x = np.concatenate(global_x)
        model = XGBRegressor(n_estimators=1000, max_depth=8, eta=0.2, subsample=1.0,
                             eval_metric="mae", early_stopping_rounds=100)
        train_idx = np.random.choice(len(global_y), size=int(0.8 * len(global_y)), replace=False)
        val_idx = [i for i in range(len(global_y)) if i not in train_idx]
        model.fit(global_x[train_idx], global_y[train_idx],
                  eval_set=[(global_x[val_idx], global_y[val_idx])],
                  verbose=False
                 )
        self.xgboost = model


    def predict(self, eval_trace_df, use_global=True):
        predictions = dict()
        labels = dict()
        for i, rows in eval_trace_df.groupby("query_idx"):
            if i not in self.isolated_rt_cache or len(rows) < 2:
                continue
            label = rows["runtime"].values
            labels[i] = label
            if self.use_train:
                concur_info = rows["concur_info_train"].values
            else:
                concur_info = rows["concur_info"].values
            query_feature = np.zeros((len(rows), self.k))
            query_feature[:, i] = self.isolated_rt_cache[i]

            concur_query_feature = np.zeros((len(rows), self.k))
            for j in range(len(rows)):
                for c in concur_info[j]:
                    if c[0] in self.isolated_rt_cache:
                        concur_query_feature[j, c[0]] += self.isolated_rt_cache[c[0]]
                    else:
                        concur_query_feature[j, c[0]] += 2
            x = np.concatenate((query_feature, concur_query_feature), axis=1)
            if self.use_pre_exec_info:
                pre_exec_query_feature = np.zeros((len(rows), self.k))
                pre_exec_info = rows["pre_exec_info"].values
                for j in range(len(rows)):
                    for c in pre_exec_info[j]:
                        if c[0] in self.isolated_rt_cache:
                            pre_exec_query_feature[j, c[0]] += self.isolated_rt_cache[c[0]]
                            #concur_query_feature[j, c[0]] += 1
                        else:
                            pre_exec_query_feature[j, c[0]] += 1
                x = np.concatenate((x, pre_exec_query_feature), axis=1)
            #if i == 0:
             #   for k in range(len(label)):
              #      print(x[k], label[k])
                
            pred = self.xgboost.predict(x)
            pred = np.maximum(pred, 0.001)
            predictions[i] = pred
        return predictions, labels



In [55]:
xgb = XGBoostPredictor(k=240)
xgb.train(concurrency_df.iloc[train_idx], isolated_trace_df=isolated_trace_df, use_pre_exec_info=True)
predictions_xgb, labels = xgb.predict(eval_trace_df)
print("===========Performance for XGBoost model=============")
result_overall_xgb, result_per_query_xgb = xgb.evaluate_performance(eval_trace_df)

===========Performance for XGBoost model=============
50% absolute error is 14.591375231742857, q-error is 3.0174608576856734
90% absolute error is 87.94467430114746, q-error is 54.69824422318488
95% absolute error is 139.6430326223371, q-error is 242.2389511445629


In [56]:
xgb = XGBoostPredictor(k=240)
xgb.train(concurrency_df.iloc[train_idx], use_train=False, isolated_trace_df=isolated_trace_df, use_pre_exec_info=True)
predictions_xgb, labels = xgb.predict(eval_trace_df)
#predictions_xgb, labels = xgb.predict(concurrency_df.iloc[train_idx], use_train=False)
print("===========Performance for XGBoost model (train on full)=============")
result_overall_xgb, result_per_query_xgb = xgb.evaluate_performance(eval_trace_df)
#result_overall_xgb, result_per_query_xgb = xgb.evaluate_performance(concurrency_df.iloc[train_idx], use_train=False)

===========Performance for XGBoost model (train on full)=============
50% absolute error is 9.593931913375854, q-error is 2.605207308176676
90% absolute error is 64.77609083652494, q-error is 476.7287504736639
95% absolute error is 99.9914370179175, q-error is 1584.1335855509935


In [58]:
i = 2
print(isolated_trace_df["runtime"].iloc[i])
idx = np.argsort(predictions_xgb[i])
np.stack((predictions_xgb[i][idx], labels[i][idx]), axis=1)

30.594415426254272


array([[ 24.47721672,  29.65713382],
       [ 39.16339874,  32.8007195 ],
       [ 48.92878342,  15.02686834],
       [ 51.34540558,  28.79204464],
       [ 55.74154282,  37.51449251],
       [ 56.58010483,  59.44050884],
       [ 64.99420929, 117.07447243],
       [ 66.40361023,  38.36281562],
       [ 74.95996094,  17.51707649],
       [ 76.66431427,  15.16912842],
       [ 79.66140747,  39.41789246],
       [ 80.12242889,  97.64560699],
       [ 81.90921021, 125.13271761],
       [ 85.38250732, 313.79030538],
       [ 85.90837097,  74.86731243],
       [ 86.91490936, 115.90412045],
       [ 97.63804626,  90.72829199],
       [102.28017426,  97.8118701 ],
       [108.31420135, 129.61169887],
       [108.3237381 ,  87.44319916],
       [108.95044708, 152.726794  ],
       [124.74686432, 127.44224906],
       [125.43645477, 326.53663707],
       [137.92114258,  99.37300205],
       [142.65739441, 176.56727529],
       [144.18693542,  90.25279713],
       [152.94996643,  19.88781929],
 

In [60]:
i = 2
print(isolated_trace_df["runtime"].iloc[i])
idx = np.argsort(predictions_xgb[i])
np.stack((predictions_xgb[i][idx], predictions_sfc[i][idx], labels[i][idx]), axis=1)

30.594415426254272


array([[ 24.47721672,  54.81194661,  29.65713382],
       [ 39.16339874,  29.73828392,  32.8007195 ],
       [ 48.92878342,  54.81194661,  15.02686834],
       [ 51.34540558,  54.81194661,  28.79204464],
       [ 55.74154282,  79.88560931,  37.51449251],
       [ 56.58010483,  54.81194661,  59.44050884],
       [ 64.99420929,  54.81194661, 117.07447243],
       [ 66.40361023, 117.49610335,  38.36281562],
       [ 74.95996094,  67.34877796,  17.51707649],
       [ 76.66431427,  67.34877796,  15.16912842],
       [ 79.66140747,  92.42244066,  39.41789246],
       [ 80.12242889, 117.49610335,  97.64560699],
       [ 81.90921021,  92.42244066, 125.13271761],
       [ 85.38250732, 104.959272  , 313.79030538],
       [ 85.90837097, 155.10659739,  74.86731243],
       [ 86.91490936, 142.56976605, 115.90412045],
       [ 97.63804626, 142.56976605,  90.72829199],
       [102.28017426, 192.71709144,  97.8118701 ],
       [108.31420135, 180.18026009, 129.61169887],
       [108.3237381 , 155.10659

In [51]:
for t, m in zip(eval_trace_df.query(f"query_idx == {i}")["runtime"].values, eval_trace_df.query(f"query_idx == {i}")["concur_info_train"].values):
    print(t, m)

189.86344814300537 [(220, 13627.95221, 13679.435213139495), (22, 13655.821968, 13690.308793942993), (18, 13659.829635, 13695.983608817825)]
39.24779963493347 [(219, 18741.014544, 19017.320929040285), (172, 18766.330489, 18823.193407138504), (159, 18785.076494, 18833.19611789946)]
12.576472997665403 [(2, 22235.128267999997, 22383.084063288083)]
174.01957988739014 [(103, 23039.037527, 23989.871220265915), (84, 23777.60402, 23806.880481362838), (225, 23782.454592, 23807.069357167235)]
273.28928780555725 [(69, 6619.045245, 6701.262070723648), (183, 6624.842624, 6721.185475400375)]
57.34541177749634 [(2, 15344.375888, 15384.13872588681), (159, 15383.91379, 15409.991699469605)]
66.66565442085266 [(49, 19109.565077, 19186.22303145442), (224, 19166.915509, 19201.158049121077)]
17.260338306427002 [(22, 36309.529222, 36351.065623748655), (103, 36319.539158, 36924.1611012354), (210, 36331.241556, 36335.6824091189)]
29.356624603271484 [(187, 39420.536466, 39892.225743410505), (48, 39584.453833, 39

In [52]:
eval_trace_df.head(5)

,index,query_idx,runtime,start_time,end_time,pre_exec_info,concur_info,num_concurrent_queries,concur_info_train,num_concurrent_queries_train
2,2,135,3.479030,0.887088,4.366118,[],"[(29, 0.0, 110.210542678833), (143, 0.0, 3.958...",2,"[(29, 0.0, 110.210542678833), (143, 0.0, 3.958...",2
15,15,202,3.515862,81.851482,85.367344,"[(28, 7.382493, 22.881574134796146), (86, 26.2...","[(29, 0.0, 110.210542678833), (184, 28.56629, ...",4,"[(29, 0.0, 110.210542678833), (184, 28.56629, ...",3
21,21,224,16.836552,100.908025,117.744577,"[(123, 43.362241999999995, 52.878114001647944)...","[(29, 0.0, 110.210542678833), (221, 85.104832,...",7,"[(29, 0.0, 110.210542678833), (221, 85.104832,...",2
27,27,67,3.878542,118.184558,122.063100,"[(75, 6.449771, 69.67815859422302), (229, 55.7...","[(221, 85.104832, 134.55309516833495), (2, 106...",3,"[(221, 85.104832, 134.55309516833495), (2, 106...",3
31,31,188,0.721914,140.729550,141.451464,"[(12, 81.43333100000001, 82.86316703477479), (...","[(2, 106.568961, 184.1101613993988), (103, 129...",3,"[(2, 106.568961, 184.1101613993988), (103, 129...",3


In [ ]:
import os
import re
dir = "/Users/ziniuw/Desktop/research/Data/join-order-benchmark/"
all_queries = []
i = 0
name_map = dict()
for file in os.listdir(dir):
    if str(file)[0].isnumeric():
        name_map[i] = file.split(".sql")[0]
        file = os.path.join(dir, file)
        with open(file, "r") as f:
            sql = f.read()
        sql = re.sub(r"\s+", ' ', sql)
        all_queries.append(sql)
        i += 1

In [ ]:
len(all_queries)

In [ ]:
with open(os.path.join(dir, "imdb_job.sql"), "w") as f:
    for q in all_queries:
        f.write(q + "\n")

In [ ]:
name_map

In [40]:
np.concatenate((np.ones(2), [1]))

array([1., 1., 1.])